In [60]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#kaggle.com/code/kritanjalijain/apriori-algorithm-hotel-management-system

In [61]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [62]:
# load data:
df_ht = pd.read_csv("hotel_bookings.csv")

df_ht.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [63]:
# Replace missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If none given, it was most likely private.
# rest schould be self-explanatory.


nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
df_clean = df_ht.fillna(nan_replacements)

# "meal" contains values "Undefined", which is equal to SC.
df_clean["meal"].replace("Undefined", "SC", inplace=True)

# Some rows contain entreis with 0 adults, 0 children and 0 babies. 
# I'm dropping these entries with no guests.
zero_guests = list(df_clean.loc[df_clean["adults"]
                   + df_clean["children"]
                   + df_clean["babies"]==0].index)

df_clean.drop(df_clean.index[zero_guests], inplace=True)


#
df_clean = df_clean[df_clean['is_canceled'] == 0]

df_clean.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [64]:
# adr 0 이하 이상값으로 판단하여 제거
df_clean = df_clean[df_clean['adr'] > 0]

''''''
#1
df_clean = df_clean[df_clean['adults'] < 5]

#+ 2 수행을 위한 총 인원변수 추가
    #+ 10인 이상의 이상값 제거
df_clean['guestNum'] = df_clean['adults'] + df_clean['babies'] + df_clean['children']
df_clean = df_clean[df_clean['guestNum'] < 10]
#2
df_clean['adr_1'] = df_clean['adr'] // df_clean['guestNum']

    #2+
df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
df_clean = df_clean.dropna()


#3 adr_1 이 0보다 작거나 같은 경우, 무의미한 데이터로 간주하여 삭제한다. & 400이상인 경우 이상치로 간주하여 제거
df_clean = df_clean[df_clean['adr_1'] > 0]
df_clean = df_clean[df_clean['adr_1'] < 400]

''''''

# adr 구간화
labels = ['low','middle','high']
df_clean['adr_bin'] = pd.qcut(df_clean['adr'],3,labels=labels)
df_clean.head()

# adr_1 구간화
labels = ['low','middle','high']
df_clean['adr_1_bin'] = pd.qcut(df_clean['adr_1'],3,labels=labels)
df_clean.head()

# guestNum 구간화
labels_g = ['1g','couple','3g','4g','5g']
df_clean['guestNum_bin'] = pd.cut(df_clean['guestNum'],5,labels=labels_g)
df_clean.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,guestNum,adr_1,adr_bin,adr_1_bin,guestNum_bin
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,75.0,0,0,Check-Out,2015-07-02,1.0,75.0,low,high,1g
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,75.0,0,0,Check-Out,2015-07-02,1.0,75.0,low,high,1g
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,98.0,0,1,Check-Out,2015-07-03,2.0,49.0,middle,middle,couple
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,98.0,0,1,Check-Out,2015-07-03,2.0,49.0,middle,middle,couple
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,107.0,0,0,Check-Out,2015-07-03,2.0,53.0,middle,middle,couple


In [65]:
# portugal_df = df_clean[df_clean["country"] == 'PRT']
# france_df = df_clean[df_clean["country"] == 'FRA']
# england_df = df_clean[df_clean["country"] == 'GBR']
# germany_df = df_clean[df_clean["country"] == 'DEU']
# italy_df = df_clean[df_clean["country"] == 'ITA']

df_clean = pd.get_dummies(data=df_clean,
               columns=['hotel','meal','market_segment','distribution_channel','country','arrival_date_year','customer_type','adr_1_bin','guestNum_bin'],
               drop_first=False,
               dtype=None
               )
df_clean.columns.values

array(['is_canceled', 'lead_time', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults',
       'children', 'babies', 'is_repeated_guest',
       'previous_cancellations', 'previous_bookings_not_canceled',
       'reserved_room_type', 'assigned_room_type', 'booking_changes',
       'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'guestNum',
       'adr_1', 'adr_bin', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'meal_SC',
       'market_segment_Aviation', 'market_segment_Complementary',
       'market_segment_Corporate', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'distribution_channel_Corporate',
       'distribution_channel_Direct'

In [75]:
clean_france_df = df_clean[[ 
        'agent',
        # 'company',
       'children', 
       'babies', 
       'previous_cancellations',
       'previous_bookings_not_canceled',  
       'booking_changes', 
       'required_car_parking_spaces', 
       'total_of_special_requests',
       'hotel_City Hotel', 'hotel_Resort Hotel',
       'country_FRA','country_PRT','country_GBR','country_ESP','country_DEU',
       'meal_BB', 'meal_FB','meal_HB', 'meal_SC',
        'market_segment_Aviation','market_segment_Complementary', 
        # 'market_segment_Corporate',
    #    'market_segment_Direct',
       'market_segment_Groups',
    #    'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS',
    #    , 'distribution_channel_TA/TO',
       'adr_1_bin_low','adr_1_bin_middle','adr_1_bin_high',
    #    'arrival_date_year_2015','arrival_date_year_2016',
        'customer_type_Group',
        'customer_type_Transient',
        'customer_type_Contract',
        'customer_type_Transient-Party',
        'guestNum_bin_1g','guestNum_bin_couple','guestNum_bin_3g','guestNum_bin_4g','guestNum_bin_5g',
       ]]
#df_clean.dropna(axis=0, subset =)
clean_france_df.dropna()

,agent,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,required_car_parking_spaces,total_of_special_requests,hotel_City Hotel,hotel_Resort Hotel,...,adr_1_bin_high,customer_type_Group,customer_type_Transient,customer_type_Contract,customer_type_Transient-Party,guestNum_bin_1g,guestNum_bin_couple,guestNum_bin_3g,guestNum_bin_4g,guestNum_bin_5g
2,0.0,0.0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,0,0,0
3,304.0,0.0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,1,0,0,0,0
4,240.0,0.0,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,0
5,240.0,0.0,0,0,0,0,0,1,0,1,...,0,0,1,0,0,0,1,0,0,0
6,0.0,0.0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,394.0,0.0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
119386,9.0,0.0,0,0,0,0,0,2,1,0,...,1,0,1,0,0,0,0,1,0,0
119387,9.0,0.0,0,0,0,0,0,4,1,0,...,1,0,1,0,0,0,1,0,0,0
119388,89.0,0.0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0


In [76]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0   
    if(x>= 1):
        return 1

# Encoding the datasets
clean_france_df_encoded = clean_france_df.applymap(hot_encode)
clean_france_df = clean_france_df_encoded.copy()

In [77]:
# Building the model
frq_items = apriori(clean_france_df, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
pd.set_option('display.max_row', 500)
rules.head(500)


#
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))


In [81]:
rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        (rules['consequent_len'] == 1) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.05)  
    ]

rules_filtered1.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
124,(distribution_channel_Corporate),(meal_BB),0.068604,0.769318,0.066055,0.962853,1.251567,0.013277,6.209916,1,1
12,(country_DEU),(agent),0.082136,0.843563,0.078334,0.953708,1.130571,0.009047,3.379366,1,1
146,(market_segment_Groups),(customer_type_Transient-Party),0.101965,0.249571,0.096677,0.948142,3.799092,0.071230,14.470907,1,1
16,(meal_SC),(agent),0.099839,0.843563,0.093339,0.934889,1.108262,0.009118,2.402629,1,1
6,(country_FRA),(agent),0.114844,0.843563,0.107198,0.933428,1.106530,0.010320,2.349894,1,1
14,(meal_HB),(agent),0.126686,0.843563,0.116997,0.923515,1.094779,0.010129,2.045336,1,1
2,(total_of_special_requests),(agent),0.513287,0.843563,0.468437,0.912622,1.081865,0.035447,1.790345,1,1
24,(guestNum_bin_3g),(agent),0.095901,0.843563,0.086402,0.900952,1.068031,0.005504,1.579403,1,1
0,(children),(agent),0.073319,0.843563,0.065728,0.896468,1.062716,0.003879,1.511003,1,1
18,(adr_1_bin_middle),(agent),0.341192,0.843563,0.305691,0.895950,1.062101,0.017874,1.503472,1,1


In [79]:
rules_filtered1 = rules[
        (rules['antecedent_len'] == 1) &
        (rules['consequent_len'] == 2) &
        (rules['support'] > 0.05) &
        (rules['confidence'] > 0.5) &
        (rules['lift'] > 1.15)  
    ]

rules_filtered1.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
296,(meal_SC),"(hotel_City Hotel, agent)",0.099839,0.546921,0.085230,0.853672,1.560867,0.030626,3.096319,1,2
800,(meal_SC),"(hotel_City Hotel, customer_type_Transient)",0.099839,0.428984,0.080596,0.807262,1.881799,0.037767,2.962650,1,2
460,(meal_SC),"(agent, customer_type_Transient)",0.099839,0.597809,0.080528,0.806579,1.349227,0.020843,2.079361,1,2
179,(children),"(agent, customer_type_Transient)",0.073319,0.597809,0.058492,0.797770,1.334490,0.014661,1.988777,1,2
284,(country_DEU),"(hotel_City Hotel, agent)",0.082136,0.546921,0.065442,0.796748,1.456787,0.020520,2.229147,1,2
277,(country_FRA),"(hotel_City Hotel, agent)",0.114844,0.546921,0.090531,0.788300,1.441340,0.027721,2.140187,1,2
513,(guestNum_bin_3g),"(agent, customer_type_Transient)",0.095901,0.597809,0.075104,0.783146,1.310028,0.017774,1.854666,1,2
465,(meal_SC),"(guestNum_bin_couple, agent)",0.099839,0.594075,0.075895,0.760169,1.279586,0.016583,1.692550,1,2
439,(guestNum_bin_3g),"(meal_BB, agent)",0.095901,0.629657,0.071425,0.744778,1.182830,0.011040,1.451058,1,2
173,(children),"(meal_BB, agent)",0.073319,0.629657,0.053599,0.731041,1.161014,0.007433,1.376948,1,2


In [71]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
    #   (rules['consequent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
344,"(market_segment_Groups, agent)",(customer_type_Transient-Party),0.073810,0.249571,0.071615,0.970273,3.887769,0.053195,25.244255,2,1
561,"(hotel_City Hotel, market_segment_Groups)",(customer_type_Transient-Party),0.057524,0.249571,0.055044,0.956882,3.834113,0.040687,17.404187,2,1
692,"(market_segment_Groups, meal_BB)",(customer_type_Transient-Party),0.070389,0.249571,0.066287,0.941723,3.773372,0.048720,12.876968,2,1
1267,"(meal_SC, total_of_special_requests)","(hotel_City Hotel, customer_type_Transient)",0.059882,0.428984,0.052741,0.880746,2.053098,0.027052,4.788251,2,2
1638,"(total_of_special_requests, meal_SC)","(customer_type_Transient, hotel_City Hotel, ag...",0.059882,0.380223,0.052059,0.869367,2.286467,0.029291,4.744426,2,3
611,"(country_GBR, adr_1_bin_low)",(hotel_Resort Hotel),0.061667,0.385197,0.050574,0.820110,2.129067,0.026820,3.417669,2,1
1218,"(required_car_parking_spaces, meal_BB)","(hotel_Resort Hotel, customer_type_Transient)",0.082027,0.276595,0.052373,0.638478,2.308349,0.029684,2.000999,2,2
1639,"(meal_SC, customer_type_Transient)","(hotel_City Hotel, total_of_special_requests, ...",0.083363,0.307190,0.052059,0.624489,2.032907,0.026451,1.844979,2,3
1458,"(country_PRT, adr_1_bin_low)","(meal_BB, hotel_Resort Hotel)",0.090749,0.295375,0.056148,0.618712,2.094667,0.029343,1.848011,2,2
1220,"(required_car_parking_spaces, customer_type_Tr...","(hotel_Resort Hotel, meal_BB)",0.085993,0.295375,0.052373,0.609033,2.061901,0.026972,1.802264,2,2


# ADR관련

In [72]:
rules_filtered2 = rules[ 
      (rules['antecedent_len'] == 2) &
      (rules['support'] > 0.05) &
       (rules['confidence'] > 0.5) &
       (rules['lift'] > 1.2)  
    ]

rules_filtered2.head(500)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
344,"(market_segment_Groups, agent)",(customer_type_Transient-Party),0.073810,0.249571,0.071615,0.970273,3.887769,0.053195,25.244255,2,1
429,"(meal_SC, total_of_special_requests)",(hotel_City Hotel),0.059882,0.614803,0.058069,0.969731,1.577305,0.021254,12.725989,2,1
556,"(meal_SC, customer_type_Transient)",(hotel_City Hotel),0.083363,0.614803,0.080596,0.966814,1.572559,0.029345,11.607149,2,1
561,"(hotel_City Hotel, market_segment_Groups)",(customer_type_Transient-Party),0.057524,0.249571,0.055044,0.956882,3.834113,0.040687,17.404187,2,1
783,"(meal_SC, total_of_special_requests)","(hotel_City Hotel, agent)",0.059882,0.546921,0.057238,0.955849,1.747690,0.024487,10.261996,2,2
692,"(market_segment_Groups, meal_BB)",(customer_type_Transient-Party),0.070389,0.249571,0.066287,0.941723,3.773372,0.048720,12.876968,2,1
1045,"(meal_SC, customer_type_Transient)","(hotel_City Hotel, agent)",0.083363,0.546921,0.078034,0.936080,1.711543,0.032441,7.088188,2,2
554,"(hotel_City Hotel, meal_SC)",(customer_type_Transient),0.088010,0.705579,0.080596,0.915763,1.297889,0.018498,3.495163,2,1
234,"(meal_SC, agent)",(hotel_City Hotel),0.093339,0.614803,0.085230,0.913126,1.485234,0.027845,4.433976,2,1
490,"(meal_SC, total_of_special_requests)",(customer_type_Transient),0.059882,0.705579,0.053626,0.895539,1.269226,0.011375,2.818485,2,1


In [73]:
# for i in range(0,len(dataset)):
#     transactions.append({str(dataset.values[i,j]) for j in range(0,len(dataset.columns)) if str(dataset.values[i,j]) != 'nan'})

# print('Transactions:\n')
# for transaction in transactions:
#     print(transaction)

1. customers travel and stay with children have more special requests
2. customers who travel and stay during weekdays have more special requests

In [74]:
# def init_candidates(transactions):
#     candidates = dict()
#     for transaction in transactions:
#         for item in transaction:
#             itemset = set()
#             itemset.add(item)
#             itemset = frozenset(itemset)
#             if itemset not in candidates:
#                 candidates[itemset] = 1
#             else:
#                 candidates[itemset] += 1
                
#     return candidates


# def prune_candidates(candidates,support):
#     fp = dict()
#     for key in candidates:
#         if candidates[key] >= support:
#             fp[key] = candidates[key]
    
#     return fp


# def get_count(transactions,itemset):
#     itemset = set(itemset)
#     count = 0
#     for transaction in transactions:
#         if itemset.issubset(transaction):
#             count += 1
            
#     return count


# def get_candidates(fp,transactions,fp_length):
#     candidates = dict()
#     for key1 in fp:
#         for key2 in fp:
#             if key2 != key1:
#                 itemset1 = set(key1)
#                 itemset2 = set(key2)
#                 itemset = itemset1.union(itemset2)
#                 itemset = frozenset(itemset)
#                 if itemset not in candidates and len(itemset) == fp_length:
#                     candidates[itemset] = get_count(transactions,itemset)
    
#     return candidates


# def get_itemset_length(candidates):
#     itemset_len = 0
#     for key in candidates:
#         itemset_len = len(key)
#         break
    
#     return itemset_len


# import itertools

# def get_rules(fp):
#     rules = []
#     for itemset in fp:
#         itemset = set(itemset)
#         for length in range(1,len(itemset)):
#             subsets = set(itertools.combinations(itemset, length))
#             for subset in subsets:
#                 lhs = set(subset)
#                 rhs = itemset.difference(lhs)
#                 rules.append([lhs,rhs])
    
#     return rules


# def prune_rules(rules,confidence,transactions):
#     fp = []
#     for rule in rules:
#         lhs = rule[0]
#         rhs = rule[1]
#         lhs_rhs = lhs.union(rhs)
#         lhs_count = get_count(transactions,lhs)
#         lhs_rhs_count = get_count(transactions,lhs_rhs)
#         conf = lhs_rhs_count/lhs_count
#         if conf >= confidence:
#             support = lhs_rhs_count/len(transactions)
#             fp.append([lhs,rhs,support,conf])
    
#     return fp

# def display(candidates,fp,length):
#     print()
#     print("****** {}-Frequent Candidates ******".format(length))
#     for key in candidates:
#         print(set(key),' : ',candidates[key])
#     print()
#     print("****** {}-Frequent Itemlist ******".format(length))
#     for key in fp:
#         print(set(key),' : ',fp[key])
# def apriori(transactions,support,confidence):
#     support = support/100 * len(transactions)
#     confidence = confidence/100
#     candidates = init_candidates(transactions)
#     old_fp = prune_candidates(candidates,support)
#     final_fp = dict()
#     display(candidates,old_fp,1)
    
#     while True:
#         new_fp_length = get_itemset_length(candidates) + 1
#         candidates = get_candidates(old_fp,transactions,new_fp_length)
#         new_fp = prune_candidates(candidates,support)
#         display(candidates,new_fp,new_fp_length)
#         if len(new_fp)<1:
#             final_fp = old_fp
#             break
#         else:
#             old_fp = new_fp
    
#     rules = get_rules(final_fp)
#     fp = prune_rules(rules,confidence,transactions)
#     return fp
# support = float(input('Enter support percentage : '))
# confidence = float(input('Enter confidence percentage : '))
# fp = apriori(transactions,support,confidence)